In [16]:
#import libs
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
lb_make=LabelEncoder()


In [17]:
#inti
lb_make=LabelEncoder()
np.random.seed(42)
tf.random.set_seed(42)

In [25]:
pwd

'C:\\adversarial_analysis-master\\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\\XAAE-IIoT\\CDAE_DAE'

In [26]:
#INput
file_name='../df_testing.csv'
missing_values = ["n/a", "na", "Infinity", "NaN","nan","-","excel","?","#DIV/0!","aza"]
df = pd.read_csv("../df_training.csv", na_values = missing_values, engine='python', skipinitialspace=True)
df=df.fillna(df.median())
df = df[df['class3'] == 0]

In [27]:
normal_data = df
features = normal_data.drop('class3', axis=1).values

# Apply MinMaxScaler to normalize the features between 0 and 1
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
X_train = torch.from_numpy(scaled_features).float()

In [28]:
# Step 3: Define the architecture of the denoising autoencoder
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(78, 36),
            nn.ReLU(),
            nn.Linear(36, 18),
            nn.ReLU(),
            nn.Linear(18, 3),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 18),
            nn.ReLU(),
            nn.Linear(18, 36),
            nn.ReLU(),
            nn.Linear(36, 78),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [29]:
# Step 4: Create an instance of the Denoising Autoencoder
dae = DenoisingAutoencoder()

# Step 5: Set the number of training epochs, learning rate, and other hyperparameters
num_epochs = 10
learning_rate = 0.001
batch_size = 32
corruption_ratio = 0.2

# Step 6: Define the reconstruction loss function (mean squared error) and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(dae.parameters(), lr=learning_rate)

In [31]:
# Step 7: Training loop
# Step 4: Create an instance of the Denoising Autoencoder
dae = DenoisingAutoencoder()

# Step 5: Set the number of training epochs, learning rate, and other hyperparameters
num_epochs = 10
learning_rate = 0.001
batch_size = 32
corruption_ratio = 0.2

# Step 6: Define the reconstruction loss function (mean squared error) and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(dae.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # Shuffle and batch the training samples
    indices = torch.randperm(X_train.size(0))
    shuffled_X_train = X_train[indices]
    batches = shuffled_X_train.split(batch_size, dim=0)

    for batch in batches:
        # Corrupt the input batch by adding noise
        corrupted_batch = batch + torch.randn_like(batch) * corruption_ratio

        # Forward pass
        output = dae(corrupted_batch)

        # Compute the reconstruction loss
        loss = criterion(output, batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [33]:
# Step 8: Apply the trained DAE to denoise the adversarial samples
adversarial_data = pd.read_csv(file_name)
adversarial_features = adversarial_data.drop('class3', axis=1).values
X_adv = torch.from_numpy(adversarial_features).float()
denoised_output = dae(X_adv)

# Step 9: Convert the denoised output tensor back to a NumPy array and rescale
denoised_samples = denoised_output.detach().numpy()
#denoised_samples = scaler.inverse_transform(denoised_samples)

# Step 10: Create a DataFrame with the denoised samples and assign the label 'class3'
denoised_data = pd.DataFrame(denoised_samples, columns=adversarial_data.columns[:-1])
denoised_data['class3'] = adversarial_data['class3']

denoised_data.to_csv('df_testing_DAE.csv', index=False)

In [44]:
# Step 8: Apply the trained DAE to denoise the adversarial samples
adversarial_data = pd.read_csv('../../df_training.csv')
adversarial_features = adversarial_data.drop('class3', axis=1).values
X_adv = torch.from_numpy(adversarial_features).float()
denoised_output = dae(X_adv)

# Step 9: Convert the denoised output tensor back to a NumPy array and rescale
denoised_samples = denoised_output.detach().numpy()
#denoised_samples = scaler.inverse_transform(denoised_samples)

# Step 10: Create a DataFrame with the denoised samples and assign the label 'class3'
denoised_data = pd.DataFrame(denoised_samples, columns=adversarial_data.columns[:-1])
denoised_data['class3'] = adversarial_data['class3']

denoised_data.to_csv('df_training_DAE.csv', index=False)

In [41]:
cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\CDAE_DAE\Cleaned_DAE_IIoT

C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\CDAE_DAE\Cleaned_DAE_IIoT


In [35]:
dae = DenoisingAutoencoder()
dae.eval()


DenoisingAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=78, out_features=36, bias=True)
    (1): ReLU()
    (2): Linear(in_features=36, out_features=18, bias=True)
    (3): ReLU()
    (4): Linear(in_features=18, out_features=3, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=18, bias=True)
    (1): ReLU()
    (2): Linear(in_features=18, out_features=36, bias=True)
    (3): ReLU()
    (4): Linear(in_features=36, out_features=78, bias=True)
    (5): Sigmoid()
  )
)

In [36]:
cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\ON

C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\ON


In [38]:
# List of CSV files
csv_files = ['FSGM-XAAE-IIoT.csv', 'BIM-XAAE-IIoT.csv', 'DF-XAAE-IIoT.csv', 'JSMA-XAAE-IIoT.csv', 'CW2-XAAE-IIoT.csv', 'CWinf-XAAE-IIoT.csv']
file_name
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

for file_name in csv_files:
    # Load the adversarial sample data
    adversarial_data = pd.read_csv(file_name)
    
    # Separate features and labels
    adversarial_features = adversarial_data.iloc[:, :-1]
    labels = adversarial_data.iloc[:, -1]

    
    # Scale features
    features_scaled = scaler.fit_transform(adversarial_features)
    
    # Convert scaled features to tensor
    X_adv = torch.from_numpy(features_scaled).float()
    
    # Apply the DAE model to denoise
    with torch.no_grad():
        denoised_output = dae(X_adv)
    
    # Convert the denoised output tensor back to a NumPy array
    denoised_samples = denoised_output.detach().numpy()
    
    # Optionally, if you want to inverse transform the denoised data to its original scale:
    # denoised_samples = scaler.inverse_transform(denoised_samples)
    
    # Create a DataFrame with the denoised samples
    denoised_data = pd.DataFrame(denoised_samples, columns=adversarial_features.columns)
    denoised_data.iloc[:, -1] = labels
    
    # Construct output file name and save the denoised DataFrame to CSV
    output_file_name = 'DAE_' + file_name
    denoised_data.to_csv(output_file_name, index=False)


In [39]:
'''
# Load the adversarial sample data
adversarial_data = pd.read_csv('train-KDD.csv')

# Separate features and labels
adversarial_features = adversarial_data.drop('42 Labels', axis=1).values

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the adversarial features
features_scaled = scaler.fit_transform(adversarial_features)

# Convert scaled features to PyTorch tensor
X_adv = torch.from_numpy(features_scaled).float()
denoised_output = dae(X_adv)

# Step 9: Convert the denoised output tensor back to a NumPy array and rescale
denoised_samples = denoised_output.detach().numpy()
#denoised_samples = scaler.inverse_transform(denoised_samples)

# Step 10: Create a DataFrame with the denoised samples and assign the label 'class3'
denoised_data = pd.DataFrame(denoised_samples, columns=adversarial_data.columns[:-1])
denoised_data['42 Labels'] = adversarial_data['42 Labels']

denoised_data.to_csv('train-DAE-KDD.csv', index=False)'''

"\n# Load the adversarial sample data\nadversarial_data = pd.read_csv('train-KDD.csv')\n\n# Separate features and labels\nadversarial_features = adversarial_data.drop('42 Labels', axis=1).values\n\n# Initialize the MinMaxScaler\nscaler = MinMaxScaler()\n\n# Fit and transform the adversarial features\nfeatures_scaled = scaler.fit_transform(adversarial_features)\n\n# Convert scaled features to PyTorch tensor\nX_adv = torch.from_numpy(features_scaled).float()\ndenoised_output = dae(X_adv)\n\n# Step 9: Convert the denoised output tensor back to a NumPy array and rescale\ndenoised_samples = denoised_output.detach().numpy()\n#denoised_samples = scaler.inverse_transform(denoised_samples)\n\n# Step 10: Create a DataFrame with the denoised samples and assign the label 'class3'\ndenoised_data = pd.DataFrame(denoised_samples, columns=adversarial_data.columns[:-1])\ndenoised_data['42 Labels'] = adversarial_data['42 Labels']\n\ndenoised_data.to_csv('train-DAE-KDD.csv', index=False)"

In [45]:
 cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\CDAE_DAE

C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\CDAE_DAE


In [47]:
torch.save(dae.state_dict(), 'DAE_model.pth')